In [1]:
# Imports and libs
import os
import os.path
import pathlib

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import io
import scipy.misc
import numpy as np

from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

import tensorflow as tf
import tensorflow_hub as hub

from concurrent.futures import ThreadPoolExecutor

from collections import Counter

import ipywidgets as widgets
from IPython.display import display

import time

tf.get_logger().setLevel('ERROR')
tf.config.run_functions_eagerly(True)

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
ALL_MODELS = {
    # ImageNet datasets
    'Faster R-CNN (OpenImages)' : 'https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1',
}

In [5]:
# Load object detection lib
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as utils_ops

%matplotlib inline

In [6]:
# Path to predefined labels for the open image dataset
PATH_TO_LABELS = './tensorflow_models/research/object_detection/data/oid_v4_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [8]:
# Select model
model_display_name = 'Faster R-CNN Inception ResNet V2 1024x1024 (OpenImages)'
model_handle = ALL_MODELS[model_display_name]

print('Selected model: '+ model_display_name)
print('Model Handle at TensorFlow Hub: {}'.format(model_handle))

# Load model
hub_model = hub.load(model_handle).signatures['default']
print('Model loaded!')

Selected model: Faster R-CNN Inception ResNet V2 1024x1024 (OpenImages)
Model Handle at TensorFlow Hub: https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1
Model loaded!


In [9]:
# Load images from inputs folder
IMAGES = []
VALID_PATHS = []
running = []

# Get all valid image paths
# image_type = 'colorized_artistic' # use this for brighter/more daring colors
# image_type = 'grayscale' # use this for the original grayscale images
image_type = 'colorized_stable' # use this for most stable results
file_path = 'inputs/' + image_type + '/'
for image_path in os.listdir(file_path):
    if (image_path.endswith('.jpg')):
        # Collect image as valid path
        VALID_PATHS.append(image_path)
        
progress_load = widgets.IntProgress(min=0, max=len(VALID_PATHS), description='Loaded: ') # instantiate the bar
load_label = widgets.Label(value='0 / ' + str(len(VALID_PATHS)))
display(widgets.HBox([progress_load, load_label])) # display the bar

# Load given image into numpy array using matplotlib imload
def load_valid_image(path):
    img = mpimg.imread(file_path + path, format='jpg')
    # Reshape image array to fit specifications required by tensorflow model
    (rows, columns, channels) = img.shape
    IMAGES.append({
        'image': img.reshape((1, rows, columns, channels)),
        'filename': path,
        'detection': {}
    })
    progress_load.value += 1
    load_label.value = str(progress_load.value) + ' / ' + str(len(VALID_PATHS))

# Parallelize execution with threadpool
def run_with_Threadpool(paths):
    with ThreadPoolExecutor(max_workers=(os.cpu_count() * 5)) as executor:
        # Schedule task for each image
        running = [executor.submit(
            lambda: load_valid_image(path)
        ) for path in paths]

"""
    Since we are working with a lot of IO operations, we use Threadpool since it is much faster for IO than using Processes:
    
    Loading 20 images:
        Runtime with Processes: ~20 seconds
        Runtime with Threadpool: ~3 seconds
"""
# Load all valid images
# start_time = time.time()
# run_with_Threadpool(VALID_PATHS)
# print("--- Runtime with Threadpool: %s seconds ---" % (time.time() - start_time))

for path in VALID_PATHS:
    load_valid_image(path)

In [20]:
# running inference
# different object detection models have additional results
# all of them are explained in the documentation
progress_inference = widgets.IntProgress(min=0, max=len(IMAGES), description='Inferred: ')
inference_label = widgets.Label(value='0 / ' + str(len(IMAGES)))
display(widgets.HBox([progress_inference, inference_label]))

start_time = time.time()
for image in IMAGES:
    # Run object detection and save results
    converted = tf.image.convert_image_dtype(image['image'], tf.float32)
    detected = hub_model(converted)
    image['detection'] = {key:value.numpy() for key,value in detected.items()}
    
    progress_inference.value += 1
    inference_label.value = str(progress_inference.value) + ' / ' + str(len(IMAGES))
    
runtime = ((time.time() - start_time) / 60)
print("--- Runtime: " + f'{runtime:.2f}' + " minutes ---")

--- Runtime: 3.03 minutes ---


In [21]:
# Copy image to draw detected objects
IMAGES_WITH_DETECTION = []
for image in IMAGES:
    IMAGES_WITH_DETECTION.append({
        'image': image['image'].copy(),
        'filename': image['filename'],
        'detection': image['detection']
    })

In [22]:
# Draw detections on copied image
for image in IMAGES_WITH_DETECTION:
    viz_utils.visualize_boxes_and_labels_on_image_array(
          image['image'][0],
          image['detection']['detection_boxes'],
          (image['detection']['detection_class_labels']).astype(int),
          image['detection']['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          # Change this if confidence score should be higher or lower
          min_score_thresh=.50,
          line_thickness=6,
          agnostic_mode=False,
          font_size=42
    )

interactive(children=(IntSlider(value=0, description='i', max=99), Output()), _dom_classes=('widget-interact',…

In [ ]:
# Display images with detection
@widgets.interact(i=(0, len(IMAGES_WITH_DETECTION)))
def show_image_at(i=0):
    plt.figure(figsize=(24,32))
    plt.imshow(IMAGES_WITH_DETECTION[i]['image'][0])
    plt.show()

In [23]:
# Add output sub-directory named after the model that was used (if it doesn't already exist)
output_dir = 'outputs/' + model_display_name + '/' + image_type + '/'
pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)

progress_save = widgets.IntProgress(min=0, max=len(IMAGES_WITH_DETECTION), description='Saved: ')
save_label = widgets.Label(value='0 / ' + str(len(IMAGES_WITH_DETECTION)))
display(widgets.HBox([progress_save, save_label]))

# Save images
for image in IMAGES_WITH_DETECTION:
    plt.imsave(output_dir + image['filename'], image['image'][0], format='jpg')
    
    progress_save.value += 1
    save_label.value = str(progress_save.value) + ' / ' + str(len(IMAGES_WITH_DETECTION))